<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>
<br>
<h1 align=center><font size = 5>IBM Data Science Capstone - Eugene Bible</font></h1>

## Introduction

This is the Jupyter Notebook to be used for the IBM Data Science Professional Certificate course by Coursera. 

In this Jupyter notebook, neighborhoods in the city of Toronto will be explored, segmented, and clustered. 
First, the data will be scraped from the Wikipedia page and be wrangled, cleaned, and then read into a pandas dataframe in a structured format. Once structured, the neighborhoods in Toronoto will be explored and clustered.

Author: Eugene Bible

## Table of Contents
<div class="alert alert-block alert-info" style="margin-top: 20px">
<font size = 3>

[1. Segmenting and Clustering Neighborhoods in Toronto](#1.-Segmenting-and-Clustering-Neighborhoods-in-Toronto) <br>
[2. The Battle of Neighborhoods - Part 1](#2.-The-Battle-of-Neighborhoods-\--Part-1)<br>
[3. The Battle of Neighborhoods - Part 2](#3.-The-Battle-of-Neighborhoods-\--Part-2)


</font>
</div>

Import the necessary libraries:

In [3]:
import urllib.request,urllib.parse,urllib.error # Library to parse websites
from bs4 import BeautifulSoup # Library to handle web scraping

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1. Segmenting and Clustering Neighborhoods in Toronto

### Scrape the Wikipedia page to obtain the table of postal codes/boroughs/neighborhoods

Page found at: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [4]:
# Define the URL of the webpage to scrape
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# Get HTML
html = urllib.request.urlopen(url).read()
#print(html)

# Use BeautifulSoup to parse the html
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())

# Parse the table only and pull all rows
table = soup.find('table')
#print(table.prettify())

<table class="wikitable sortable">
 <tbody>
  <tr>
   <th>
    Postcode
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighbourhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Parkwoods" title="Parkwoods">
     Parkwoods
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Victoria_Village" title="Victoria Village">
     Victoria Village
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
     Downtown Toronto
    </a>
   </td>
   <td>
    <a href="

### Transform the table into a Pandas Dataframe
Requirements:
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- !!!! More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [7]:
# Scraping the rows of the table
table_rows = table.find_all('tr')

rows = []
for tr in table_rows:
    #print('count = ' + str(count))
    th = tr.find_all('th') # Grab header
    td = tr.find_all('td') # Grab row
    if th:
        row = [th.text.rstrip() for th in th]
        rows.append(row)
    else:
        row = [tr.text.rstrip() for tr in td]
        if row[1] == 'Not assigned': continue # If no borough, don't keep the row
        if row[2] == 'Not assigned': row[2] = row[1] # If neighborhood is not assigned, set it to the Borough
        rows.append(row)  
            
# Assign list of rows to a Pandas DF
nhScraped_df = pd.DataFrame(rows)

# Name the columns after the first row (headers) then drop the header row
nhScraped_df.columns = nhScraped_df.iloc[0]
nhScraped_df = nhScraped_df.drop(nhScraped_df.index[0])
nhScraped_df.head(10)


,Postcode,Borough,Neighbourhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Downtown Toronto,Queen's Park
7,M9A,Queen's Park,Queen's Park
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern
10,M3B,North York,Don Mills North


In [12]:
# Now that we have the 'Not Assigned' entries taken care of, need to combine the rows with the same Postcode but different neighborhoods

# Use groupby to sort the list by Postcode then Borough
nhGrouped_df = nhScraped_df.groupby(['Postcode','Borough'])
print(nhGrouped_df)

for key, item in nhGrouped_df:
    print(nhGrouped_df.get_group(key), "\n\n")

0 Postcode      Borough Neighbourhood
8      M1B  Scarborough         Rouge
9      M1B  Scarborough       Malvern 


0  Postcode      Borough   Neighbourhood
21      M1C  Scarborough  Highland Creek
22      M1C  Scarborough      Rouge Hill
23      M1C  Scarborough      Port Union 


0  Postcode      Borough Neighbourhood
33      M1E  Scarborough     Guildwood
34      M1E  Scarborough   Morningside
35      M1E  Scarborough     West Hill 


0  Postcode      Borough Neighbourhood
39      M1G  Scarborough        Woburn 


0  Postcode      Borough Neighbourhood
43      M1H  Scarborough     Cedarbrae 


0  Postcode      Borough        Neighbourhood
54      M1J  Scarborough  Scarborough Village 


0  Postcode      Borough         Neighbourhood
66      M1K  Scarborough  East Birchmount Park
67      M1K  Scarborough               Ionview
68      M1K  Scarborough          Kennedy Park 


0  Postcode      Borough Neighbourhood
79      M1L  Scarborough      Clairlea
80      M1L  Scarborough   Gold

In [14]:
# Join the grouped entries
neighborhoods_df = nhGrouped_df['Neighbourhood'].apply(', '.join)
neighborhoods_df.head()

Postcode  Borough    
M1B       Scarborough                            Rouge, Malvern
M1C       Scarborough    Highland Creek, Rouge Hill, Port Union
M1E       Scarborough         Guildwood, Morningside, West Hill
M1G       Scarborough                                    Woburn
M1H       Scarborough                                 Cedarbrae
Name: Neighbourhood, dtype: object

Case1
Case1
Case1
Case3
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
Case1
[['Postcode', 'Borough', 'Neighbourhood'], ['Postcode', 'Borough', 'Neighbourhood'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Harbourfront'], ['M6A', 'North York', 'Lawrence Heights'], ['M6A', 'North York', 'Lawrence Manor'], ['M7A', 'Downtown Toronto', "Queen's Park"], ['M9A', "Queen's Park", "Queen's Park"], ['M1B', 'Scarborough', 'Rouge'], ['M1B', 'Scarborough', 'Malvern'], ['M3B', 'North York', 'Don Mills North'], ['M4B', 'East York', 'Woodbine Gardens'], [

,Postcode,Borough,Neighbourhood
1,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M9A,Queen's Park,Queen's Park
9,M1B,Scarborough,Rouge
10,M1B,Scarborough,Malvern




3. To create the dataframe:

4. Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

The package is so popular that there is a plethora of tutorials and examples on how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:


Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:


Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:


Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:


Important Note: There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (2 marks)

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.
Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

In [1]:
import pandas as pd
import numpy as np

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## 2. The Battle of Neighborhoods - Part 1

## 3. The Battle of Neighborhoods - Part 2